In [11]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from textblob import TextBlob
import itertools 

In [ ]:
comments = pd.read_excel(r"C:\sentimentanalysis_comments.xlsx")
comments

In [24]:
pd.set_option('display.max_colwidth', None)

In [3]:
comments.dropna(inplace=True)

In [ ]:
# create a list out of all comments
comments_list = comments['reason_behind_score'].tolist()
print(comments_list)

In [5]:
# passing the list of comments through a tokenization function
# utilizing a list of stopwords, removing punctuation, applying porter stemmer
stoplist = set(stopwords.words('english'))
punctuation = string.punctuation + '‘’“”' 
ps = PorterStemmer()

def preprocessed_comments(comment_list):
    preprocessed_comments = []
    for comment in comment_list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        tokens = word_tokenize(comment)
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        tokens = [ps.stem(token) for token in tokens]
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        preprocessed_comments.append(tokens)
    return preprocessed_comments

In [ ]:
preprocessed_comments_list = preprocessed_comments(comments_list)
print(preprocessed_comments_list)

In [34]:
# Perform sentiment analysis and add sentiment labels to the dataframe
comments['Sentiment_polarity'] = comments['reason_behind_score'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [38]:
# Perform sentiment analysis and add sentiment labels to the dataframe
comments['Sentiment_subjectivity'] = comments['reason_behind_score'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [35]:
# function to set thresholds per sentiment score intervals
def get_sentiment_label(sentiment_score):
    if sentiment_score <= -0.8:
        return 'Highly Negative'
    elif sentiment_score <= -0.6:
        return 'Negative 0.6-0.8'
    elif sentiment_score <= -0.4:
        return 'Negative 0.4 to 0.6'
    elif sentiment_score <= -0.2:
        return 'Moderately negative 0.2 to 0.4'
    elif sentiment_score < 0:
        return 'Slightly negative (0 to 0.2)'
    elif sentiment_score == 0:
        return 'Neutral'
    elif sentiment_score < 0.2:
        return 'Slightly positive (0 to 0.2)'
    elif sentiment_score < 0.4:
        return 'Moderately positive (0.2 to 0.4)'
    elif sentiment_score < 0.6:
        return 'Positive (0.4 to 0.6)'
    elif sentiment_score < 0.8:
        return 'Positive (0.6 to 0.8)'
    else:
        return 'Highly Positive'

In [40]:
# Add sentiment label column to the dataframe
comments['Sentiment Label Polarity'] = comments['Sentiment_polarity'].apply(get_sentiment_label)
comments['Sentiment Label Subjectivity'] = comments['Sentiment_subjectivity'].apply(get_sentiment_label)

In [ ]:
comments

In [ ]:
negative_scores_comments = comments.loc[comments['Sentiment_polarity'] < 0.0]
negative_scores_comments

In [ ]:
positive_scores_comments = comments.loc[comments['Sentiment_polarity'] > 0.0]
positive_scores_comments

In [ ]:
neutral_scores_comments = comments.loc[comments['Sentiment_polarity'] == 0.0]
neutral_scores_comments

In [ ]:
positive_scores_comments2 = comments.loc[comments['Sentiment_subjectivity'] > 0.0]
positive_scores_comments2

In [47]:
# Create an empty dictionary to store the counts for each sentiment bucket
sentiment_counts = {}

# Iterate over the sentiment scores in the dataframe
for sentiment_score in comments['Sentiment_polarity']:
    # Get the sentiment label for the score
    sentiment_label = get_sentiment_label(sentiment_score)
    
    # Increment the count for the sentiment label in the dictionary
    sentiment_counts[sentiment_label] = sentiment_counts.get(sentiment_label, 0) + 1

# Print the counts for each sentiment bucket
for sentiment_label, count in sentiment_counts.items():
    print(f"{sentiment_label}: {count} comments")


Slightly negative (0 to 0.2): 615 comments
Moderately positive (0.2 to 0.4): 562 comments
Neutral: 815 comments
Positive (0.4 to 0.6): 345 comments
Moderately negative 0.2 to 0.4: 427 comments
Slightly positive (0 to 0.2): 655 comments
Highly Negative: 101 comments
Negative 0.6-0.8: 113 comments
Negative 0.4 to 0.6: 263 comments
Positive (0.6 to 0.8): 274 comments
Highly Positive: 251 comments


In [48]:
# Iterate over the sentiment scores in the dataframe
for sentiment_score in comments['Sentiment_subjectivity']:
    # Get the sentiment label for the score
    sentiment_label = get_sentiment_label(sentiment_score)
    
    # Increment the count for the sentiment label in the dictionary
    sentiment_counts[sentiment_label] = sentiment_counts.get(sentiment_label, 0) + 1

# Print the counts for each sentiment bucket
for sentiment_label, count in sentiment_counts.items():
    print(f"{sentiment_label}: {count} comments")


Slightly negative (0 to 0.2): 615 comments
Moderately positive (0.2 to 0.4): 1211 comments
Neutral: 1490 comments
Positive (0.4 to 0.6): 1517 comments
Moderately negative 0.2 to 0.4: 427 comments
Slightly positive (0 to 0.2): 808 comments
Highly Negative: 101 comments
Negative 0.6-0.8: 113 comments
Negative 0.4 to 0.6: 263 comments
Positive (0.6 to 0.8): 1433 comments
Highly Positive: 864 comments
